<h1>Assignment 4</h1>
<h2>Pima Indians Diabetes</h2>
<br>
<p>I am interested in this dataset because my father has suffered with Diabetes since he was a child.</p>
<p>https://www.kaggle.com/uciml/pima-indians-diabetes-database</p>
    

<h4>Imports</h4>

In [1]:
from sklearn.cluster import KMeans
import numpy as np
import warnings

### Exercise 1: Clustering, Bisecting k-Means

<h4>Helper Functions</h4>

In [2]:
def feature_normalization(X):
    # compute mean and stdev over axis 0, the feature vector (down the column)
    mean = np.mean(X,0)
    stddev = np.std(X,0)
    
    # elementwise difference
    diff = np.subtract(X,mean)
    
    # elementwise division
    normalized = np.divide(diff,stddev)
    
    # for testing
    # for each feature, stddev should be 1 and mean should be 0
    #print("stddev of normalized", np.std(normalized,0))    
    #print("mean of normalized", np.mean(normalized,0))    
    
    return normalized  

In [3]:
def kmeans_2_clusters(X, max_iter, random_state = 42):
    km = KMeans(n_clusters=2, max_iter = max_iter, random_state=random_state)
    km.fit(X)
    return km.labels_

#### BKMEANS FUNCTION

In [4]:
def bkmeans(X, k, max_iter, random_state = 42):
    # K should be greater than 1, otherwise it can be considered already a cluster
    assert k>1
    
    # Step 1, initialize by dividing in to two clusters
    clusters = kmeans_2_clusters(X, max_iter)
    cluster_counter = 2
    
    # Step 2, Iterate until reached num of clusters desired
    while cluster_counter < k:
        # Get largest cluster
        cluster_ids, cluster_counts = np.unique(clusters, return_counts = True)
        largest_cluster_index = np.argmax(cluster_counts)
        
        # This is actually unnecessary because the id always matches the index
        largest_cluster_id = cluster_ids[largest_cluster_index]
        
        # Get indices of elements in the largest cluster 
        indices = np.nonzero(clusters == largest_cluster_id)[0]
        
        # Get values of subcluster for use in Kmeans
        values_largest_cluster = X[indices]
        
        # Split subcluster into two smaller clusters
        subcluster_labels = kmeans_2_clusters(values_largest_cluster, max_iter)
        
        # Get indices of new subcluster that is mapped to 0 (arbitrary, can choose 0 or 1)
        subcluster_label_indices = np.nonzero(subcluster_labels == 0)[0]

        # Map indices of new subcluster back to the main cluster array
        original_indices = indices[subcluster_label_indices]
        
        # Update main cluster array with new subcluster assignments
        clusters[original_indices] = cluster_counter
        cluster_counter += 1
    
    return clusters
    

In [5]:
def ex1():
    # load data
    data = np.loadtxt('./diabetes.csv',delimiter=',',skiprows=1)
    X = data[:, 0:-1]
    #y = data[:, -1]

    # Normalize Data
    Xn = feature_normalization(X)
    
    # Get Clusters
    clusters = bkmeans(Xn, 10, 10)
    
    print("A4, EX1\nclusters.shape",clusters.shape,"\nclusters",clusters)

In [6]:
#ex1()

# Exercise 2: Non-linear Dimensionality Reduction

<p>The algorithm can be implemented with gradient descent using the following steps:</p>
<ol><li>Start with a random two-dimensional layout Y of points (Y is a n × 2 matrix).</li>
    <li>Compute the stress E of Y . See slide 47 of Lecture 12 for the formula.</li>
    <li>If E < ε, or if the maximum number of iterations iter has been reached, stop.</li>
    <li>For each yi of Y , find the next vector yi(t + 1) based on the current yi(t). See slide 48 of lecture 12</li><li>Go to Step 2.</li></ol>


In [7]:
def get_distance_matrix(X, Y):
    """ Create distance matrix 
    Note:
        1. Optimization level - highly optimized! I used this during assignment 1. 
        Algorithm was found https://medium.com/@souravdey/l2-distance-matrix-vectorization-trick-26aa3247ac6c
        It is a no loop solution, which means it can handle the matrices in
        one line of code rather than having to iterate over the rows of one
        matrix
    """
    
    print("get_distance_matrix")
    print(X.shape)
    print(Y.shape)
    
    # perform operation in parts due to tiny numbers causing error with sqrt
    eu1 = -2 * np.dot(Y, X.T) + np.sum(X**2, axis=1) + np.sum(Y**2, axis=1)[:, np.newaxis]
    
    # replace tiny numbers with 0
    eu1 = np.where(np.absolute(eu1)<1e-10,0,eu1)
    
    # return sqrt of 'clean' matrix
    return np.sqrt(eu1)

def get_c(X):
    # C is the sum of all elements of the upper triangular distance matrix
    return np.sum(np.triu(X))

In [20]:
   
def sammon(hd_matrix, max_iter=2, eps=.3, alpha=0):
    hd_delta_matrix = get_distance_matrix(hd_matrix, hd_matrix)
    print("hd_delta_matrix",hd_delta_matrix)

    c = get_c(hd_delta_matrix)
    print("c",c)
    
    reduced_matrix = np.random.rand(len(hd_matrix),2)
    print("reduced_matrix",reduced_matrix)
    
    for curr_iter in range(1,max_iter):
        print("i", curr_iter)

        yitp1 = gradient(reduced_matrix, hd_delta_matrix, eps, c)
        
        #print("yitp1",yitp1)
        #print("yitp1-reduced_matrix",yitp1-reduced_matrix)
        
        #reduced_matrix = yitp1
        
    pass

def gradient(reduced_matrix, hd_delta_matrix, eps, c ):
    reduced_delta_matrix = get_distance_matrix(reduced_matrix, reduced_matrix)
    a = reduced_matrix - eps
    b = pd1(reduced_matrix, hd_delta_matrix, reduced_delta_matrix, c)
    pass
    
def pd1(reduced_matrix, hd_delta_matrix, reduced_delta_matrix, c):
    a = (-2/c)
    print("a", a)
    
    b = hd_delta_matrix - reduced_delta_matrix
    print("b", b)
    
    cc = hd_delta_matrix * reduced_delta_matrix
    print("cc", cc)
    
    # We should not be dividing by zero, however, this causes a runtime warning only 
    # on Jupyter and only affects the elements along the diagonal (or maybe other datapoints with 0 distance)
    # Therefore, this operation does not block an otherwise efficient way to compute this as a matrix operation
    # We simply perform the division, ignore the warning in place, and then fix the divide by zero cells, changing
    # them from nan back to 0. Then we can sum over the matrix later using numpy and avoid for loops
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        try:
            d = b / c
            print("d", d)
        except Warning as e:
            print('error found:', e)
    
    e = np.nan_to_num(d)
    print("e",e)
    
    f = (-2/c) * np.sum(e) * reduced_matrix
    print("f",f)
    
    return f

def pd2(reduced_matrix, hd_delta_matrix, reduced_delta_matrix, c):
    return (-2/c) * np.sum()

def ex2():
    print("A4, EX2")
    
    # load data
    data = np.loadtxt('./diabetes.csv',delimiter=',',skiprows=1)
    X = data[:, 0:-1]
    X = data[:5,0:3]
    #y = data[:, -1]

    # Normalize Data
    #Xn = feature_normalization(X)
    Xn = X
    
    # Get Clusters
    sammon(Xn)
    
    
ex2()

A4, EX2
get_distance_matrix
(5, 3)
(5, 3)
hd_delta_matrix [[ 0.         63.48228099 35.95830919 59.51470407 34.36568055]
 [63.48228099  0.         98.27003612  4.         58.14636704]
 [35.95830919 98.27003612  0.         94.28149341 52.49761899]
 [59.51470407  4.         94.28149341  0.         54.59853478]
 [34.36568055 58.14636704 52.49761899 54.59853478  0.        ]]
c 555.1150251553903
reduced_matrix [[0.95030622 0.29950058]
 [0.27768875 0.15929752]
 [0.04230412 0.87012085]
 [0.23844364 0.06102602]
 [0.54449739 0.47736873]]
i 1
get_distance_matrix
(5, 2)
(5, 2)
a -0.0036028569023873044
b [[ 0.         62.79520665 34.88589355 58.76395881 33.92260281]
 [62.79520665  0.         97.52125329  3.89418192 57.73120907]
 [34.88589355 97.52125329  0.         93.44896404 51.86008287]
 [58.76395881  3.89418192 93.44896404  0.         54.08180453]
 [33.92260281 57.73120907 51.86008287 54.08180453  0.        ]]
cc [[ 0.         43.61704644 38.56225317 44.680382   15.22666809]
 [43.61704644  0. 

# Exercise 3: Visualization of Results

A4, EX2
get_distance_matrix
(5, 3)
(5, 3)
hd_delta_matrix [[ 0.         63.48228099 35.95830919 59.51470407 34.36568055]
 [63.48228099  0.         98.27003612  4.         58.14636704]
 [35.95830919 98.27003612  0.         94.28149341 52.49761899]
 [59.51470407  4.         94.28149341  0.         54.59853478]
 [34.36568055 58.14636704 52.49761899 54.59853478  0.        ]]
c 555.1150251553903
reduced_matrix [[0.45532847 0.19196157]
 [0.53508728 0.57765903]
 [0.05354567 0.08765681]
 [0.82661492 0.48836397]
 [0.50378633 0.44228082]]
i 1
get_distance_matrix
(5, 2)
(5, 2)
a -0.0036028569023873044
b [[ 0.         63.08842314 35.54320812 59.03961669 34.1107141 ]
 [63.08842314  0.         97.58302484  3.69510334 58.00741737]
 [35.54320812 97.58302484  0.         93.41074528 51.92449175]
 [59.03961669  3.69510334 93.41074528  0.         54.27243363]
 [34.1107141  58.00741737 51.92449175 54.27243363  0.        ]]
cc [[ 0.         25.00299455 14.9263327  28.27468472  8.76209579]
 [25.00299455  0. 